### open csv files from directory and create markdown tables

- get csv files from the local build input/image/source folder
- print out markdown files
- save files as input/includes files
- !check out how I made excel files...

**IF EXCEL IS SOURCE USE CONVERT TO CSV**

In [ ]:
from pathlib import Path
import pandas as pd
import csv
from json import loads
from IPython.display import Markdown, display

In [ ]:
# get path from IG output dir
in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')
# in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes/ballot-intro.csv')
out_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out')

in_path

### load CSV using pandas

trim all strings too


In [ ]:
df = pd.read_csv(in_path)
df.fillna('-', inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df

In [ ]:
for i,col in enumerate(df.columns):
    print(i,col)

In [ ]:
for i in df.index:
    print(i,df.loc[i,'Data Element'])

In [ ]:
def editor_warning(my_file):
  return f'''<!-- {my_file}
*****************************************************************************************************
*                                  WARNING: DO NOT EDIT THIS FILE                                   *
*                                                                                                   *
* This file is generated by csv_to_markdown_tabler.ipynb. Any edits you make to this file will be   *
* overwritten                                                                                       *
* To change the contents of this file, edit input/images/{in_path.name}                     *
*****************************************************************************************************
-->\n\n'''

download_link = f'''The data element mapping table is available as a [CSV](data-element-mapping.csv) and [Excel](data-element-mapping.xlsx) file.'''

# convert to markdown table 
# select columns:

maps = {#'data_elements_to_submit' : ([0,1,3,4,5,7,8,10,11],[1,4,5,6,8]),
        'attachments_to_275' : ([0,1,3,4,5,7,8,10,11],[1,6,4,5,8]),
        #'attachments_to_requests_277_278' : ([0,1,3,4,5,7,8,10,11],[5,6,2,3]),
        'attachments_to_requests' : ([0,1,3,4,5,7,8,10,11],[1,6,7]),
        #'data_elements_to_requests': (None,[1,6,7]),
        
        'requests-277_278': (None,[1,7,2,3,9])
        }
# this maps highlights new stuff in columns 2-5, using columns 10-13 as the source
maps = {
        'attachments_to_275' : ([0,1,3,4,5,7,8,10,11],[1,6,12,13,8]),
        'attachments_to_requests' : ([0,1,3,4,5,7,8,10,11],[1,6,7]),
        'q-attachments_to_requests' : ([0,1,3,4,5,7,8,10,11,15,16,],[1,6,7]),
        'requests-277_278': (range(0,15),[1,7,10,11,9])
        }
# maps = {'text' : ([0,1,3,4,5,7,8,10,11],[1,6,5]) }
# maps = {'text' : (None,[0,1,2]) }

# df1 = df[['No','Data Element','CDex $submit-attachment Parameter', 'X12n 275']]
df1 = df.iloc[:, [0,1,5,4,7]]
for k,v in maps.items():
  if k != 'text':

    my_file =f'{k}.md'
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    with open(out_path / my_file, 'w') as md:
      my_table = df1.to_markdown(index=False, tablefmt="github")
      for i in range(1,10):
        my_table = my_table.replace('----','---')
        my_table = my_table.replace('  ',' ')
      print(f'======== my_file: {my_file} =======')
      display(Markdown(my_table))
      md.write(f'{editor_warning(my_file)}{my_table}\n{{:.grid}}\n\n{download_link}')
  else:
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    my_table = df1.to_markdown(index=False, tablefmt="github")
    for i in range(1,10):
      my_table = my_table.replace('----','---')
      my_table = my_table.replace('  ',' ')
    print(my_table)
